In [ ]:
import os
project_root = os.path.dirname(os.getcwd())
import sys
# Add the directory to the Python path
sys.path.append(f"{project_root}/src")

In [ ]:
from pathlib import Path
import shutil

import ray

from config import gpt2_cfg as cfg 
from preprocessor.datasource_processor import DatasourceProcessor
from preprocessor.chunk_processor import ChunkProcessor
from preprocessor.token_processor import TokenProcessor

In [ ]:
def generate_chunk_tokn_ids(cfg,source_file_paths,split):
    file_path_ds = ray.data.from_items(source_file_paths)
    datasource_processor = DatasourceProcessor(source_format="parquet")
    texts_ds = file_path_ds.map(datasource_processor,
                            num_cpus=2,
                            concurrency=2)

    tokenizer_class = TokenProcessor.create(cfg['ray_data']['tokenizer_class']['name'])
    tokenizer_args =  cfg['ray_data']['tokenizer_class']['args']
    tokenizer= tokenizer_class(**tokenizer_args)
    tokens_ds = texts_ds.map(tokenizer,
                               num_cpus=4,
                               concurrency=4)
    block_size = cfg["model"]["block_size"]
    stride = cfg["model"]["stride"]
    chunk_processor = ChunkProcessor(block_size=block_size, stride=stride)
    chunked_tokens_ds = tokens_ds.flat_map(chunk_processor,
                                  num_cpus=4,
                                  concurrency=4)
    target_path = Path(cfg["dataset"]["chunked_tokens"]).joinpath(split)

    if os.path.exists(target_path):
        shutil.rmtree(target_path)
        print(f"The directory {target_path} has been removed.")
    else:
        print(f"The directory {target_path} does not exist.")

    chunked_tokens_ds.write_parquet(target_path,concurrency=8)


In [ ]:
source_path = Path(cfg["dataset"]["path"])
source_file_paths = list(source_path.glob("*.parquet"))

len = len(source_file_paths)

train_ratio = cfg["ray_data"]["train_ratio"]

train_file_paths = source_file_paths[:int(len*train_ratio)]
validate_file_paths = source_file_paths[int(len*train_ratio):]

generate_chunk_tokn_ids(cfg,train_file_paths,"train")
generate_chunk_tokn_ids(cfg,validate_file_paths,"validate")



(raylet) *** SIGTERM received at time=1725327154 on cpu 14 ***
(raylet) PC: @     0x7418dbafa5b4  (unknown)  __open
(raylet)     @     0x7418dba28520  (unknown)  (unknown)
(raylet) PC: @     0x7da93462188d  (unknown)  syscall
(raylet) [2024-09-03 01:32:34,871 E 683310 683310] logging.cc:440: *** SIGTERM received at time=1725327154 on cpu 14 ***
(raylet) [2024-09-03 01:32:34,871 E 683310 683310] logging.cc:440: PC: @     0x7418dbafa5b4  (unknown)  __open
(raylet) [2024-09-03 01:32:34,871 E 683310 683310] logging.cc:440:     @     0x7418dba28520  (unknown)  (unknown)
[2024-09-03 01:34:34,044 E 558795 560784] core_worker.cc:661: :info_message: Attempting to recover 408 lost objects by resubmitting their tasks. To disable object reconstruction, set @ray.remote(max_retries=0).


(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/opt/conda/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 709, in <module>
    monitor.run()
  File "/opt/conda/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 584, in run
    self._run()
  File "/opt/conda/lib/python3.11/site-packages/ray/autoscaler/_private/monitor.py", line 438, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)

